In [83]:
import mysql.connector as mysql
from mysql.connector import Error
import pandas as pd
import numpy as np

In [84]:
dataset = pd.read_csv("Airplane_Crashes_and_Fatalities_Since_1908.csv")

In [85]:
db_name='airplane_crashes'
try:
    mydb = mysql.connect(host='localhost', user='root', password='franci22') # you can add the auth_plugin here too (ref line 26)
    if mydb.is_connected():
        mycursor = mydb.cursor()
        mycursor.execute('SHOW DATABASES')
        result = mycursor.fetchall()
        print(result)
        for x in result:
            if db_name == x[0]:
                mycursor.execute('DROP DATABASE ' + db_name) # delete old database
                mydb.commit() # make the changes official
                print("The database already exists! The old database has been deleted!)")

        mycursor.execute("CREATE DATABASE "+ db_name)
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

[('airplane_crashes',), ('electronics',), ('example_database',), ('GoShare',), ('information_schema',), ('mid_practice',), ('movies',), ('mysql',), ('parents',), ('performance_schema',), ('sys',)]
The database already exists! The old database has been deleted!)
Database is created


In [86]:
mycursor.execute('USE' + ' ' + db_name)

In [87]:
#create tables
mycursor.execute(
      '''
        CREATE TABLE `Airplane` (
          `serial_number` varchar(100),
          `registration` varchar(100),
          `operator` varchar(100),
          `type` varchar(100),
          PRIMARY KEY (serial_number, registration)
        );
      '''
      )

In [88]:
mycursor.execute(
      '''
        CREATE TABLE Flight (
           flight_number varchar(100),
           aboard int,
           route varchar(100),
           serial_number varchar(100),
           registration varchar(100),
           PRIMARY KEY (flight_number),
           FOREIGN KEY (`serial_number`, `registration`) REFERENCES `Airplane` (`serial_number`, `registration`)
        );
      '''
      )

In [89]:
mycursor.execute(
    '''
        CREATE TABLE `Crash` (
          `index` int,
          `date` date,
          `time` time,
          `location` varchar(100),
          `summary` varchar(2000),
          `serial_number` varchar(100),
          `registration` varchar(100),
          PRIMARY KEY (`index`),
          FOREIGN KEY (`serial_number`, `registration`) REFERENCES `Airplane`(`serial_number`, `registration`)
        );
    '''
)

In [90]:
mycursor.execute(
    '''
    CREATE TABLE `Fatalities` (
    `crash_index` int,
    `number_of_fatalities` int,
    `ground` int,
    PRIMARY KEY (`crash_index`),
    FOREIGN KEY (`crash_index`) REFERENCES `Crash`(`index`)
);
    '''
)

In [91]:
selection = dataset.iloc[3196:] #take values from 1980-90 and take all columns except from the first
null_val = selection.isnull().sum()
selection = selection.drop(labels='Flight #', axis=1)

In [92]:
import random

flight_number = []

while len(flight_number) != 2072:
    n = random.randint(1000, 10000)
    if n not in flight_number:
        flight_number.append(n)


selection.insert(6, 'Flight#', flight_number)



serial_num = selection.loc[3196:, 'cn/In']
serial_num_ = list(serial_num)
selection = selection.drop(labels='cn/In', axis=1)

for x in range(len(serial_num_)):
    n = random.randint(100, 8000)
    if serial_num_[x] is np.nan:
        while n in serial_num:
            n = random.randint(100, 8000)
        serial_num_[x] = str(n)

selection.insert(9, 'Serial#', serial_num_)


registration = selection.loc[3196:, 'Registration']
registration_ = list(registration)
selection = selection.drop(labels='Registration', axis=1)

import string
for r in range(len(registration_)):
    reg1 = ''.join(random.choice(string.ascii_uppercase) for x in range(2))
    reg2 = ''.join(random.choice(string.ascii_uppercase) for x in range(3))
    reg = reg1+'-'+reg2
    if reg not in registration_:
        registration_[r] = reg

import datetime
from datetime import datetime

times = selection['Time'].tolist()
t2 = []
for t in times:
    if type(t)!=str:
            t2.append(0)
    else:
        allowed_characters = ['0','1','2','3','4','5','6','7','8','9',':']
        for i in t:
            if i not in allowed_characters or (i == ':' in (t[0] or t[-1])):
                t = t.replace(i,'')
        if ':' not in t:
            t = t[:len(t)//2]+':'+t[len(t)//2:]
        p = t.partition(':')
        if len(p[0]) > 2:
            t = t[1:]
        if len(p[2]) > 2:
            t = t[:-1]
        try:
            time = datetime.strptime(t, '%H:%M').time()
            t2.append(time)
        except:
            t2.append(0)
            continue
selection = selection.drop(labels='Time', axis=1)
selection.insert(2, 'Time', t2)




date = selection.loc[3196:, 'Date']
date_ = list(date)
datelist = [datetime.strptime(x,'%m/%d/%Y').strftime('%Y/%m/%d') for x in date_]
#print(list)
selection = selection.drop(labels='Date', axis=1)
selection.insert(1, 'Date', datelist)



TypeError: 'list' object is not callable

In [ ]:
datairplane = selection[['Serial#', 'Registration', 'Operator', 'Type']]

In [ ]:
selection.head(2000)

In [ ]:
for i,row in datairplane.iterrows():
    sql = "INSERT INTO airplane_crashes.Airplane VALUES (%s,%s,%s,%s)"
    mycursor.execute(sql, tuple([row['Serial#'], row['Registration'], row['Operator'], row['Type']]))
    mydb.commit()

In [ ]:
#mycursor.execute("SELECT * FROM Airplane")
#result = mycursor.fetchall()
#print(result)

In [ ]:
dataflight = selection[['Flight#', 'Aboard', 'Route', 'Serial#', 'Registration']]

In [ ]:
for i, row in dataflight.iterrows():
    sql = "INSERT INTO airplane_crashes.Flight VALUES (%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple([row['Flight#'], row['Aboard'], row['Route'], row['Serial#'], row['Registration']]))

    mydb.commit()

In [ ]:
datacrash = selection[['index', 'Date', 'Time', 'Location', 'Summary', 'Serial#', 'Registration']]

In [ ]:
for i, row in datacrash.iterrows():
    sql = "INSERT INTO airplane_crashes.Crash VALUES (%s,%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple([row['index'], row['Date'], row['Time'], row['Location'],  row['Summary'], row['Serial#'], row['Registration']]))

    mydb.commit()

In [ ]:
datafatalities = selection[['index', 'Fatalities', 'Ground']]
for i, row in datafatalities.iterrows():
    sql = "INSERT INTO airplane_crashes.Flight VALUES (%s,%s,%s)"
    mycursor.execute(sql, tuple([ row['index'], row['Fatalities'], row['Ground']]))

    mydb.commit()